In [1]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import sys
import codecs
import mpld3

In [2]:
# initially we need document folders and document id's
path_to_dataset = '/home/madhi/Documents/python programs/neuralnetworks/fp/Reuters21578-Apte-115Cat/training'
folder_list = os.listdir(path_to_dataset)

In [3]:
#make a dictionary of document category and documents contain in the folder
cat_and_docs = [] #list
document_folder = [] #list
for docs_folder in folder_list:
    folder_path = path_to_dataset + "/" + docs_folder
    list_of_documents = os.listdir(folder_path)
    for real_docs in list_of_documents:
        document_folder.append(real_docs)
    #append folder_name and docs into the same list
    cat_and_docs.append((docs_folder,document_folder))
    #empty the document_folder list
    document_folder = []
    
print(cat_and_docs[1]) #the list is in format category and number of docs in that list

('palladium', ['0001792', '0001862'])


In [4]:
#make into dictionary
cate = [] 
documents = []

for category,docs in cat_and_docs:
    cate.append(category)
    documents.append(docs)
    
#convert into dictionary
total_dataset = dict(zip(cate,documents))

In [5]:
len(total_dataset)

116

In [6]:
feature_df = pd.read_csv('features.txt',sep='\t',header=None,names=['doc_id','features'])
feature_df.head()

,doc_id,features
0,0,"alfonsin,allsuite,arnott,bonded,briefed,bulax,..."
1,0,"dependent,cellulosa,allentown,bases,ergenc,err..."
2,0,"cellulosa,emphasizing,edina,celsius,dependent,..."
3,0,"edina,cellulosa,allentown,emphasizing,celsius,..."
4,0,"downed,ecologists,edina,emphasizing,cellulosa,..."


In [7]:
doc_category = feature_df['doc_id']
features = feature_df['features']
#convert into pandas Series
features_id = pd.Series(doc_category)
features_series = pd.Series(features)

print(features_id.shape,features_series.shape)

(11475,) (11475,)


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
#in features_series list all features is considered as a single string
#we need to sepearate the words and have to apply tf-idf vectorizer and ngram to each words
def document_category(features_id):
    docs_category = []
    for ids in features_id:
        docs_category.append(ids)
    return docs_category

def separate_features_the_series(feature_series):
    main_feature_series = []
    features_stripped = []
    for features in feature_series:
        data_features = features.replace(","," ")
        main_feature_series.append(data_features)
    
    return main_feature_series

docs_id = document_category(features_id)
features_splitted = separate_features_the_series(features_series)
print(len(features_splitted))

11475


In [9]:
#tokenize and stem the words
import nltk
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

In [10]:
def tokenize_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]',token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

def tokenize_only(text):
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]',token):
            filtered_tokens.append(token)
    return filtered_tokens

In [11]:
tfidf_vectorizer = TfidfVectorizer(max_df=1.0,max_features=20000,use_idf=True,tokenizer=tokenize_and_stem,ngram_range=(1,3))
%time  tfidf_matrix = tfidf_vectorizer.fit_transform(features_splitted)
print(tfidf_matrix.shape)

CPU times: user 7.54 s, sys: 8 ms, total: 7.55 s
Wall time: 7.78 s
(11475, 14085)


In [12]:
terms = tfidf_vectorizer.get_feature_names()

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

dist = 1-cosine_similarity(tfidf_matrix)

In [14]:
dist.shape

(11475, 11475)

In [15]:
len(cate)

116

In [16]:
len(documents)

116

In [17]:
from sklearn.externals import joblib
from sklearn.cluster import KMeans

num_clusters = 50
km = KMeans(n_clusters = num_clusters)

%time km.fit(tfidf_matrix)
#dump the clusters for later user
#joblib.dump(km,'clusters.pkl')

CPU times: user 13.2 s, sys: 40 ms, total: 13.2 s
Wall time: 13.4 s


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=50, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [18]:
#load the cluster
from sklearn.externals import joblib
cl_load = joblib.load('clusters.pkl')
def_clusters = cl_load.labels_.tolist()
print(def_clusters[:96]) #just print to see the clusters

[2, 23, 23, 23, 23, 23, 47, 23, 23, 21, 1, 21, 21, 23, 21, 21, 47, 21, 23, 23, 21, 21, 3, 3, 3, 21, 21, 23, 3, 3, 3, 11, 3, 3, 23, 21, 21, 3, 3, 47, 21, 47, 10, 11, 1, 28, 21, 47, 3, 47, 1, 3, 11, 1, 21, 41, 1, 21, 28, 1, 28, 28, 47, 28, 1, 11, 28, 27, 27, 47, 47, 11, 28, 11, 47, 1, 27, 28, 3, 27, 27, 11, 23, 11, 27, 27, 36, 47, 28, 36, 47, 28, 27, 36, 48, 43]


In [19]:
#convert features_splitted into a Series Pandas
document = {'category':docs_id,'content':features_splitted ,'clusters':def_clusters}
frame = pd.DataFrame(document,index=[def_clusters],columns=['category','content','clusters'])

In [20]:
frame['clusters'].value_counts()

15    360
32    345
45    345
19    330
3     300
14    300
16    300
22    300
28    285
41    285
37    285
38    285
6     270
17    270
8     270
49    270
4     270
47    270
46    255
21    255
1     255
23    255
33    240
26    240
24    225
42    225
35    225
39    225
43    225
12    225
13    225
5     225
27    210
10    210
9     210
31    195
36    195
25    180
44    180
11    180
20    165
2     165
18    165
34    165
29    135
48    120
0     120
40     90
30     75
7      75
Name: clusters, dtype: int64

In [21]:
frame.iloc[9658:9668] #the less number of clusters the more number of same category documents belongs to same clusters

,category,content,clusters
37,87,dynalectron athletic elections bhputah denwa c...,37
8,87,detected conservatism bonn decided converts da...,8
17,87,coopervision clercq chatted enfield employee e...,17
41,87,cbl dwindled educate danger academics amnount ...,41
42,87,athletic alatenn cry counties arbitrary compac...,42
1,87,clercq discuss abating august completion brant...,1
32,87,athletic bale cole darman audit applications b...,32
37,87,ameritrust clipbecause airframe dnzr column be...,37
5,87,cbl breathing ecuadoreans culver dur clock cru...,5
10,87,cbl csx bni awaits creditcard coop creates dec...,10


In [22]:
grouped = frame['category'].groupby(frame['clusters'])
grouped.mean() 

clusters
0     50.325000
1     46.874510
2     48.793939
3     46.530000
4     49.351852
5     50.484444
6     47.696296
7     51.253333
8     49.174074
9     47.452381
10    47.947619
11    46.572222
12    51.017778
13    50.035556
14    47.390000
15    47.825000
16    50.840000
17    48.840741
18    51.921212
19    49.651515
20    51.848485
21    45.631373
22    47.343333
23    45.729412
24    52.035556
25    49.283333
26    50.533333
27    46.185714
28    46.312281
29    46.777778
30    51.600000
31    51.184615
32    49.562319
33    50.254167
34    50.181818
35    50.262222
36    46.989744
37    48.821053
38    49.266667
39    47.240000
40    50.911111
41    48.245614
42    48.484444
43    47.048889
44    51.288889
45    48.211594
46    49.850980
47    45.988889
48    50.008333
49    50.318519
Name: category, dtype: float64

In [23]:
#convert document into series
series_docs = pd.Series(documents)
individual_docs = [] #to store individual documents
for document in series_docs:
    for indi_docs in document:
        individual_docs.append(indi_docs)
print(len(individual_docs))

11476


In [24]:
series_indi_docs = pd.Series(individual_docs)
series_indi_docs = series_indi_docs[:-1]
print(series_indi_docs.shape) #the shape of frame is (11475,4)

(11475,)


In [25]:
frame.head()

,category,content,clusters
2,0,alfonsin allsuite arnott bonded briefed bulax ...,2
23,0,dependent cellulosa allentown bases ergenc err...,23
23,0,cellulosa emphasizing edina celsius dependent ...,23
23,0,edina cellulosa allentown emphasizing celsius ...,23
23,0,downed ecologists edina emphasizing cellulosa ...,23


In [26]:
frame['document'] = series_indi_docs.values

In [27]:
frame.iloc[6000:6010]

,category,content,clusters,document
40,44,discussions alter cyl countertrading deregulat...,40,0003569
16,44,beaverton compo briefly downers brsa dec consu...,16,0008869
12,44,complied augment discuss deadbeats ecuadoreans...,12,0003770
0,44,boxboard entering advocate divide consultancy ...,0,0007668
12,44,electromedics committees ctws effectuated cara...,12,0005203
31,44,ameritrust bonas concerns docugraphix cry enti...,31,0001490
26,44,ctws cyl chartleston discontinued dangerous be...,26,0008716
26,44,docugraphix dark boxboard chauvco bottler augm...,26,0008699
34,44,braude dardanelles alter communities amtr answ...,34,0008809
31,44,briefly cyl dark cara cigra ctws bluebellaltam...,31,0001533


In [28]:
document_content = frame['content']
stopwords = nltk.corpus.stopwords.words('english')

In [29]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in document_content:
    allwords_stemmed = tokenize_and_stem(i)
    totalvocab_stemmed.append(allwords_stemmed)
    
    allwords_tokens = tokenize_only(i)
    totalvocab_tokenized.append(allwords_tokens)

In [30]:
len(totalvocab_stemmed)

11475

In [31]:
dict_documents = {'WordTokenized':totalvocab_tokenized,'WordStemmed':totalvocab_stemmed}

In [32]:
vocab_frame = pd.DataFrame({'words':totalvocab_tokenized})

In [33]:
terms[6040:6050]

['cambist amag',
 'cambist amag conshohocken',
 'cambist cidn',
 'cambist cidn coldwel',
 'cambist cleric',
 'cambist cleric denpac',
 'cambist empow',
 'cambrian',
 'cambrian china',
 'cambrian china airdrom']

In [34]:
from __future__ import print_function
print("Top terms per Cluster")
print()
top_clusters_words = [] #to store the top cluster words
#sort clusters
order_centroids = km.cluster_centers_.argsort()[:,::-1]
for i in range(num_clusters):
    print("Clusters {} words:".format(i),end=' ')
    for ind in order_centroids[i,:25]: #top 25 words
        important_terms = terms[ind].split(' ')[0]
        print('%s' % terms[ind].split(' ')[0],end=',')
        top_clusters_words.append((i,important_terms))
    print()
    print() #add whitespace
    
    print("Clusters {} Documents:".format(i),end = ' ')
    for title in frame.ix[i]['document'].values.tolist():
        print("{}".format(title),end=' ')
    print()
    print() #add whitespace

print()
print()

Top terms per Cluster

Clusters 0 words: eightyearold,btlr,empi,durum,defin,aulnays,carniv,eightlock,entireti,denpac,conshohocken,demonstr,clorox,divis,entireti,conshohocken,clorox,empi,empi,amalgam,amalgam,entireti,aulnays,durum,durum,

Clusters 0 Documents: 0006948 0008152 0003370 0009237 0004297 0001595 0000105 0002873 0002275 0008334 0001939 0006461 0008924 0005167 0005348 0002379 0000547 0009370 0005100 0006256 0001043 0006346 0005831 0002578 0006971 0001111 0003891 0001808 0008511 0005770 0000563 0003756 0001525 0001772 0009079 0003644 0004615 0004191 0000425 0006360 0003298 0005641 0003713 0001436 0009113 0008120 0006179 0007104 0005413 0006216 0006666 0003360 0007473 0002295 0005262 0003671 0006024 0008866 0007542 0007434 0007668 0007882 0009266 0003902 0005680 0008217 0000504 0008865 0009141 0006459 0000275 0007894 0005244 0001452 0002083 0007899 0007318 0005173 0004245 0009554 0002257 0000333 0006188 0004844 0007659 0000643 0002449 0001783 0004015 0005217 0001603 0001172 0006

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


0001074 0001242 0007762 0007575 0002108 0007237 0008441 0003302 0003281 0004069 0001240 0008997 0004469 0009474 0006321 0008997 0000524 0007495 0002004 0006706 0004073 0001735 0005392 0009046 0007484 0005108 0009567 0004295 0007074 0002938 0001037 0001706 0001472 0008024 0007747 0000443 0003820 0004281 0000748 0000134 0003390 0001922 0001897 0004386 0008170 0004570 0003403 0005321 0004703 0001072 0003494 0002987 0008233 0001286 0006458 0007633 0003974 0003882 0006905 0001652 0000858 0007167 0002927 0002245 0001323 0004784 0004720 0005124 0008928 0007694 0005168 0004271 0005548 0000380 0005991 0005898 0003468 0008449 0005479 0004499 0000091 0004373 0000319 0009566 0003011 0009522 0007137 0005732 0008367 0001096 0009419 0002172 0002244 0004982 0004592 0001033 0004352 0008939 0003931 0004415 0001935 0000696 0008891 0005674 0005743 0002171 0008474 0007964 0007052 0004565 0007984 0007935 0004003 0004988 0003418 0001152 0001329 0007364 0000488 0008282 0000390 0006726 0005405 0005974 0004937 

Clusters 18 words: diamonddril,balladur,balladur,applic,afbf,buckley,engin,caan,certif,end,buckley,depend,buckley,arsenal,draft,defin,brighter,entic,editon,endtoend,berr,cornfeedm,counderwrit,casual,depend,

Clusters 18 Documents: 0003045 0000327 0006081 0003092 0006639 0003178 0003239 0003429 0006197 0000613 0009472 0008855 0000338 0002014 0002195 0006871 0001688 0003590 0001248 0007014 0000240 0005705 0001470 0005397 0009294 0003295 0004854 0004839 0005030 0000146 0003185 0009191 0003282 0001399 0005506 0006288 0001144 0004530 0005445 0005454 0001658 0005492 0007415 0004672 0001336 0002780 0000381 0003906 0004807 0007559 0005068 0007063 0000131 0004628 0000992 0005881 0006455 0008164 0004539 0002981 0005035 0002574 0003823 0002534 0009057 0002126 0001486 0007006 0003217 0001363 0007867 0002254 0007586 0006476 0003273 0007511 0002347 0001486 0009593 0007568 0008270 0008599 0001692 0009020 0008896 0001364 0009329 0008936 0007352 0005461 0005981 0002142 0007597 0008624 0006954 0000234 0

Clusters 27 Documents: 0000277 0000355 0009417 0003922 0009213 0002982 0007260 0000825 0001761 0007558 0000094 0007161 0008263 0006823 0002080 0002960 0005494 0004817 0000177 0005617 0009439 0006340 0002087 0003175 0001025 0008939 0001398 0001998 0008880 0003705 0006481 0001692 0002998 0007649 0002078 0003250 0005049 0006401 0008983 0007535 0006149 0003726 0008176 0008943 0005090 0003780 0001767 0006420 0000996 0003499 0007114 0003870 0002955 0008832 0000740 0000110 0004591 0003933 0003758 0000108 0005161 0005042 0004340 0001038 0000683 0003024 0003795 0003808 0008684 0004989 0002729 0003419 0006233 0008734 0001390 0000976 0003981 0000666 0007079 0008278 0003943 0002227 0004543 0005207 0005490 0007416 0005008 0008570 0004742 0000977 0008537 0005269 0006731 0003972 0006547 0006169 0001841 0005294 0000869 0004590 0007240 0000086 0004280 0005447 0001594 0004821 0004717 0001358 0006617 0001355 0008706 0007381 0009412 0007292 0003247 0008268 0000162 0004893 0004932 0000193 0002798 0007159 0

Clusters 35 Documents: 0002633 0000355 0002259 0006148 0006311 0006381 0000362 0007735 0008210 0002590 0003022 0003378 0000192 0004374 0006833 0006418 0001735 0002274 0000952 0008319 0005313 0008984 0005727 0004603 0002768 0006209 0008689 0009032 0000347 0006209 0004924 0008876 0007285 0009361 0009437 0006685 0009202 0006695 0004304 0006142 0003434 0002435 0003761 0004154 0006658 0008005 0006728 0005437 0004331 0003040 0003029 0006334 0008037 0007903 0007347 0001319 0005086 0006992 0000407 0004371 0004623 0003496 0006591 0001344 0003342 0004399 0001359 0005535 0008554 0006387 0007194 0002926 0005155 0002514 0006312 0006590 0002922 0007634 0001394 0009317 0007060 0006641 0006805 0000424 0004820 0004492 0003964 0001460 0005597 0008604 0001119 0001490 0009255 0009005 0000419 0008869 0004107 0006978 0001190 0001369 0001694 0003287 0008136 0000368 0006581 0009196 0003922 0000454 0001357 0006382 0006209 0006224 0005663 0007699 0002871 0005514 0007396 0003164 0006326 0008101 0008301 0008858 0

Clusters 45 Documents: 0003162 0001520 0005181 0007093 0000069 0003506 0004829 0006985 0002639 0001505 0000879 0008982 0005447 0003464 0005162 0007781 0002188 0004592 0007127 0003538 0001789 0000601 0002235 0005188 0001489 0000338 0005106 0008646 0005853 0008601 0008972 0007203 0004241 0008025 0009068 0008882 0001744 0009170 0004732 0005734 0002977 0009369 0008697 0001892 0001091 0004425 0007733 0007502 0009434 0009600 0006887 0007102 0008850 0008958 0009446 0006479 0003660 0000556 0003657 0006413 0003781 0003607 0008420 0006329 0007914 0006898 0005353 0005208 0007768 0001459 0004334 0001469 0009426 0003807 0002147 0005646 0005487 0005920 0002535 0005096 0004571 0004157 0006522 0003518 0005500 0004301 0007670 0003984 0003473 0003410 0008509 0004787 0006060 0004781 0004417 0008410 0002145 0000320 0000661 0008979 0000324 0002475 0001312 0001292 0000288 0005581 0004326 0006300 0006031 0008000 0007121 0000939 0006568 0006999 0007465 0002426 0002129 0003647 0003952 0003177 0008565 0002447 0

In [35]:
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.style.use('ggplot')

In [36]:
dist.shape

(11475, 11475)

In [37]:
#the eucledian distance shape(11475,11475)
#decompose the batch into 15 batches into shape(765,765)
#store into the list
dist_as_batch = []
i=1
previous = 0
while(i*765 <= dist.shape[0]):
    index = i*765
    dist_as_batch.append(dist[previous:index])
    previous = index+1
    i+=1

In [38]:
len(dist_as_batch[0][1]) #the matrix is in shape(765,11475)

11475

Now the cosine_similarity variable dist has shape (11475,11475) . We need to convert them into 2D array using **MultiDimensionalScaling or PCA** . If we use MDS function directly we get **Memeroy Error** . So we need to decompose the whole dist to **multiple batches** , which is in the shape (765,11475)  into 15 batches. If we need to apply **MDS.fit_transform()** the row and column of the matrix should be in the shape(765,765)

In [39]:
#convert the dist into DatFrame 
dist_dataFrame = pd.DataFrame(data=dist,index=None,)

In [40]:
dist_dataFrame.tail()

,0,1,2,3,4,5,6,7,8,9,...,11465,11466,11467,11468,11469,11470,11471,11472,11473,11474
11470,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.000000,0.985393,0.925325,0.978381,0.951615,8.881784e-16,9.730359e-01,9.661215e-01,1.000000e+00,9.731314e-01
11471,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.952356,0.928705,0.961177,0.916614,0.962638,9.730359e-01,8.881784e-16,9.754390e-01,1.000000e+00,9.556951e-01
11472,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.904297,0.916244,0.945297,0.966465,0.984209,9.661215e-01,9.754390e-01,-2.220446e-16,9.395556e-01,9.755260e-01
11473,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.955957,0.942677,0.908549,0.967765,0.947922,1.000000e+00,1.000000e+00,9.395556e-01,-8.881784e-16,9.676659e-01
11474,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.979543,0.904792,0.945820,0.958450,1.000000,9.731314e-01,9.556951e-01,9.755260e-01,9.676659e-01,-8.881784e-16


In [41]:
#divide the whole data into 765 batch
#iterate through each row and convert 11475 columns to 2 columns using PCA analysis
#store the result in new file for future purpose
#start the iteration of each row by iterrows()
#Error Faced  : Expected 2D array but got 1D array instead
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=2,n_iter=2,random_state=5)
#convert dataFrame into numpy array
dist_as_np = np.array(dist)

In [42]:
dist_as_np.shape

(11475, 11475)

In [43]:
d = [] #list
i=1
previous = 0
while(i*765 <= dist_as_np.shape[0]):
    current_index = i*765
    data = dist_as_np[previous:current_index]
    svd_vector = svd.fit_transform(data)
    d.append((i,svd_vector))
    previous = current_index
    print("Finished Processing {} batch".format(i))
    i+=1

Finished Processing 1 batch
Finished Processing 2 batch
Finished Processing 3 batch
Finished Processing 4 batch
Finished Processing 5 batch
Finished Processing 6 batch
Finished Processing 7 batch
Finished Processing 8 batch
Finished Processing 9 batch
Finished Processing 10 batch
Finished Processing 11 batch
Finished Processing 12 batch
Finished Processing 13 batch
Finished Processing 14 batch
Finished Processing 15 batch


In [44]:
#now we decomposed the high dimensional data into low 2D data
#enter the data into dataFrame
#write the 2D vectors into text file for future reference
with open('low_dimensional_data.txt','w') as fp:
    for i,array_data in d:
        for two_d_vectors in array_data:
            vectors_as_str = str(two_d_vectors)
            if vectors_as_str.startswith(' '):
                vectors_as_str = vectors_as_str.replace(' ','')
            replace_chars = ['[',']']
            for char in replace_chars:
                vectors_as_str = vectors_as_str.replace(char,'')
            fp.write(vectors_as_str)
            fp.write("\n")
        

In [45]:
#convert into dataframe
low_dim_df = pd.read_csv('low_dimensional_data.txt',sep='\s+',header=None,names=['X','Y'])

In [46]:
low_dim_df.iloc[1222:1230]

,X,Y
1222,106.620350,-0.140611
1223,106.780017,-0.072778
1224,106.770989,-0.055833
1225,106.872403,-0.014668
1226,106.731865,-0.086752
1227,106.625474,-0.153665
1228,106.657120,-0.144136
1229,106.710341,-0.084602


In [47]:
low_dim_df.shape

(11475, 2)

In [48]:
xs,ys = low_dim_df.iloc[:,0],low_dim_df.iloc[:,1]
print(xs.shape,ys.shape)

(11475,) (11475,)


In [49]:
#pick 50 colors for different clusters
i=0
colors_list = []
with open('../clustering/random_colors.txt','r') as fp:
    data = fp.read()
    #split by \n
    data_split = data.split("\n")
    for hex_colors in data_split:
        colors_list.append((i,hex_colors))
        i+=1

In [50]:
colors_list.pop()

(50, '')

In [51]:
colors_dict = dict(colors_list) #converted into dictionary
colors_dict

{0: '#e66582',
 1: '#65afb0',
 2: '#cb7fb4',
 3: '#f68956',
 4: '#f95c55',
 5: '#e0f876',
 6: '#6ddd48',
 7: '#444a68',
 8: '#90508b',
 9: '#69eb1e',
 10: '#95f05c',
 11: '#712302',
 12: '#dddb6d',
 13: '#101d48',
 14: '#0135f2',
 15: '#5b90a8',
 16: '#909236',
 17: '#fe92e5',
 18: '#0b518f',
 19: '#979efb',
 20: '#b2dd57',
 21: '#e8fbdd',
 22: '#823c4f',
 23: '#4db437',
 24: '#40281d',
 25: '#2cd7c3',
 26: '#ed8dea',
 27: '#396f0d',
 28: '#b18ab4',
 29: '#7f24ea',
 30: '#e9f8fd',
 31: '#cdab7c',
 32: '#961059',
 33: '#59c668',
 34: '#a898fe',
 35: '#e7f923',
 36: '#b38dd2',
 37: '#8f4642',
 38: '#875e6e',
 39: '#01e9ca',
 40: '#4184f3',
 41: '#85caa0',
 42: '#67ebe5',
 43: '#e38618',
 44: '#dfc2b6',
 45: '#cef5a4',
 46: '#875733',
 47: '#e9036c',
 48: '#987393',
 49: '#14e0f1'}

In [52]:
from itertools import groupby
from operator import itemgetter
clusters_title = [] #list to identify the what represents each cluster
top_clusters_words.sort(key=itemgetter(0))
groups = groupby(top_clusters_words,itemgetter(0))
words = [[items[1] for items in data] for key,data in groups]

In [53]:
len(words)

50

In [54]:
%matplotlib inline
#creating the dataFrame
docs = [] #list to store the docs
document = frame['document'] # document is in type series
for doc in document:
    docs.append(docs)

print(len(docs))

11475


In [55]:
%pylab

df = pd.DataFrame(dict(X=xs,Y=ys,label=def_clusters,title=docs))
#group by cluster
groups = df.groupby('label')
#plot
fig, ax = plt.subplots(figsize=(17, 9)) # set size
ax.margins(0.05)

#iterate through groups to  layer the plot
for name,group in groups:
    ax.plot(group.X,group.Y,marker='o',linestyle='',ms=12,color=colors_dict[name],mec='none')
    ax.set_aspect('auto')
    ax.tick_params(axis='x',which='both',bottom='off',
                  top='off',labelbottom='off')
    ax.tick_params(axis='y',which='both',bottom='off',
                  top='off',labelleft='off')
    
    ax.legend(numpoints=1) #show legend with only 1 point
    

for i in range(len(df)):
    ax.text(df.iloc[i]['X'],df.ix[i]['Y'],df.ix[i]['title'],size=8)


plt.savefig('visualization_clustering.png') #save the plot
    

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.5/dist-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['title', 'svd', 'char', 'dist']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


KeyboardInterrupt: 